## Stone Soup Data Reader
[Documentation](https://stonesoup.readthedocs.io/en/v1.4/auto_examples/readers/Custom_Pandas_Dataloader.html#sphx-glr-auto-examples-readers-custom-pandas-dataloader-py)

In [ ]:
import pandas as pd
import numpy as np
import csv
from datetime import datetime, timedelta
from importlib import reload  # Python 3.4+
from typing import Tuple
import itertools
from matplotlib import pyplot as plt
from math import ceil

import dateutil
from pymap3d import geodetic2enu

import sys
sys.path.append('C:/Users/ttrinter/git_repo/cspeed/data_common')
import data_functions as dfunc
# sys.path.append("../stonesoup") # go to parent dir
# # from customFunctions import *

from stonesoup.reader import DetectionReader, GroundTruthReader
from stonesoup.reader.pandas_reader import DataFrameDetectionReader
from stonesoup.base import Property
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, \
                                               ConstantVelocity
from stonesoup.types.detection import Detection
from stonesoup.plotter import AnimatedPlotterly, Plotter, Plotterly

from stonesoup.base import Property
from stonesoup.buffered_generator import BufferedGenerator
from stonesoup.functions import cart2sphere, sphere2cart
from stonesoup.models.measurement.linear import LinearGaussian
from stonesoup.models.measurement.nonlinear import CartesianToElevationBearingRange, \
    CartesianToBearingRange, Cartesian2DToBearing, CombinedReversibleGaussianMeasurementModel
from stonesoup.types.angle import Bearing, Elevation
from stonesoup.types.detection import Detection
from stonesoup.types.groundtruth import GroundTruthState, GroundTruthPath
from stonesoup.types.state import StateVector
from stonesoup.plotter import AnimatedPlotterly, Plotter, Plotterly

# Tracker Imports
from stonesoup.dataassociator.neighbour import GNNWith2DAssignment
from stonesoup.deleter.error import CovarianceBasedDeleter
from stonesoup.deleter.multi import CompositeDeleter
from stonesoup.deleter.time import UpdateTimeDeleter
from stonesoup.feeder.multi import MultiDataFeeder
from stonesoup.feeder.time import TimeBufferedFeeder
from stonesoup.hypothesiser.distance import DistanceHypothesiser
from stonesoup.initiator.simple import MultiMeasurementInitiator
from stonesoup.measures import Mahalanobis
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, ConstantVelocity
from stonesoup.predictor.kalman import ExtendedKalmanPredictor
from stonesoup.tracker.simple import MultiTargetTracker
from stonesoup.types.array import StateVector, CovarianceMatrix
from stonesoup.types.state import GaussianState
from stonesoup.updater.kalman import ExtendedKalmanUpdater

sensor_positions = { 'RDU103': (51.52126391, 5.85862734)}



## Get Data from BigQuery
* rdp_straight: short, straight flight path
* rdp_extended: longer flight path
* adsb_straight: truth for rdp_straight
* adsb_extended: truth for rdp_extended


In [ ]:
# adsb_sql = """SELECT `timestamp`,
#         time_of_day, 
#         latitude, 
#         longitude, 
#         target_address,
#         flight_level, 
#         rho, 
#         theta
# FROM radar_data.adsb
# WHERE test_date = '2024-07-17'
# AND target_address=10537421
# and latitude is not NULL
# AND rho<20
# ORDER BY `timestamp`"""

# adsb_straight = dfunc.query_to_df(adsb_sql)

# rdp_sql = f"""SELECT 
#         `timestamp`,
#         time_of_day,
#         cal, 
#         rho,
#         theta, 
#         x, 
#         y, 
#         field_note 
# FROM radar_data.rdp
# WHERE `timestamp` >= '{adsb_straight.timestamp.min().strftime("%Y-%m-%d %H:%M:%S")}'
# AND `timestamp` <= '{adsb_straight.timestamp.max().strftime("%Y-%m-%d %H:%M:%S")}'
# AND rho >= {adsb_straight.rho.min()-0.5}
# AND rho <= {adsb_straight.rho.max()+0.5}
# AND theta >= {adsb_straight.theta.min()- 5}
# AND theta <= {adsb_straight.theta.max()+5}"""

# rdp_straight = dfunc.query_to_df(rdp_sql)
# rdp_straight.head()

## Save/Read  to/from CSV

In [ ]:
from math import pi
data_dir = 'C:/Users/ttrinter/git_repo/Stone-Soup/data'
adsb_file = f'{data_dir}/adsb_straight.csv'
# adsb_straight.to_csv(adsb_file, index=False)
adsb_data = pd.read_csv(adsb_file)
adsb_data['timestamp'] = pd.to_datetime(adsb_data['timestamp'], errors='coerce')
adsb_data = adsb_data.loc[~adsb_data['timestamp'].isna()]
adsb_data['timestamp'] = pd.to_datetime(adsb_data['timestamp'], errors='coerce')
adsb_data['timestamp'] = adsb_data['timestamp'].dt.tz_localize(None)

rdp_file = f'{data_dir}/rdp_straight.csv'
# rdp_straight['timestamp'] = pd.to_datetime(rdp_straight['timestamp'], errors='coerce')
# rdp_straight['theta_rad'] = np.deg2rad(rdp_straight.theta)
# rdp_straight.loc[rdp_straight.theta_rad>2*pi, 'theta_rad'] = rdp_straight.loc[rdp_straight.theta_rad>2*pi, 'theta_rad'] - 2*pi 

# rdp_straight = rdp_straight.loc[~rdp_straight['timestamp'].isna()]
# rdp_straight.to_csv(rdp_file, index=False)
rdp_data = pd.read_csv(rdp_file)
rdp_data['timestamp'] = pd.to_datetime(rdp_data['timestamp'], errors='coerce')
rdp_data['timestamp'] = rdp_data['timestamp'].dt.tz_localize(None)


print(f'ADSB: {len(adsb_data)}')
print(f'RDP: {len(rdp_data)}')

In [ ]:
# adsb_data.to_csv(adsb_file, index=False)
# rdp_data.to_csv(rdp_file, index=False)

In [ ]:
print(f'rho: {rdp_data.rho.min()} - {rdp_data.rho.max()}')
print(f'theta: {rdp_data.theta.min()} - {rdp_data.theta.max()}')

In [ ]:
np.rad2deg(rdp_data.theta_rad).hist()

In [ ]:
lat0, lon0, alt0 = 38.25049, -121.92474, 40

class RDPReader(DetectionReader):
    rdp_file: str = Property(doc="File with the radar data.")
    ndim_state: int = Property(default=6)
    pos_mapping: Tuple[int, int] = Property(default=(0, 2))
    vel_mapping: Tuple[int, int] = Property(default=(1, 3))
    pos_noise_diag: Tuple[float, float] = Property(
        default=(np.radians(1) ** 2, 25 ** 2))
    vel_noise_diag: Tuple[float, float] = Property(default=(1, 1))
    min_reflection: float = Property(default=-np.inf)
    max_reflection: float = Property(default=35)

    # Kaggle Alvira Location
    # lat, lon, alt = 51.52126391, 5.85862734, 31

    # Travis Radar Location
    lat, lon, alt = 38.25049, -121.92474, 40

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        position_model = CartesianToBearingRange(
            self.ndim_state, self.pos_mapping, np.diag(self.pos_noise_diag),
            translation_offset=StateVector([*geodetic2enu(self.lat, self.lon, self.alt,
                                                          lat0, lon0, alt0)]))
        velocity_model = LinearGaussian(
            self.ndim_state, self.vel_mapping, np.diag(self.vel_noise_diag))

        self.model = CombinedReversibleGaussianMeasurementModel([position_model, velocity_model])

    @BufferedGenerator.generator_method
    def detections_gen(self):
        with open(self.rdp_file, newline='') as csv_file:
            for row in csv.DictReader(csv_file):
                if not row['timestamp']:
                    continue

                timestamp = dateutil.parser.parse(row['timestamp'], ignoretz=True)
                # lat = float(row['latitude'])
                # lon = float(row['longitude']) 
                rho = float(row['rho'])*METERS_in_NM
                phi = 2*pi - float(row['theta_rad']) + pi/2

                # we don't have these usually - commenting out
                # azimuth = np.radians(90 - float(row['az_velocity']))
                # elevation = np.radians(float(row['alt_velocity']))
                # speed = float(row['speed'])
                azimuth = 100
                elevation = 0
                speed = 20

                metadata = {
                    'cal': row['cal'],
                    'sensor': 'RDU103', 
                    'reflection': 0
                    }

                if not self.min_reflection < metadata['reflection'] < self.max_reflection:
                    continue

                # easting, northing, *_ = geodetic2enu(lat, lon, alt, self.lat, self.lon, self.alt)
                # rho, phi, _ = cart2sphere(easting, northing, alt)
                dx, dy, dz = sphere2cart(speed, azimuth, elevation)
                # dx, dy, dz = 0.5, 0.5, 0

                yield timestamp, {Detection(
                    # [Bearing(phi), rho], timestamp=timestamp,
                    [Bearing(phi), rho, dx, dy], timestamp=timestamp,
                    metadata=metadata, measurement_model=self.model)}


In [ ]:
class ADSBTruthReader(GroundTruthReader):
    adsb_file: str = Property(doc="File with the adsb data.")

    @staticmethod
    def single_ground_truth_reader(adsb_file, isset=True):
        truth = GroundTruthPath()
        with open(adsb_file, newline='') as csv_file:
            for row in csv.DictReader(csv_file):
                lat = float(row['latitude'])
                lon = float(row['longitude'])
                alt = float(row['flight_level'])*100
                time = dateutil.parser.parse(row['timestamp'])
                if row['target_address'] != "":
                    planename = row['target_address']
                x, y, z = geodetic2enu(lat, lon, alt, lat0, lon0, alt0)
                truth.append(GroundTruthState(
                    [x, 0, y, 0, z, 0],
                    timestamp=time,
                    metadata={"id": planename}))
            if isset:
                truth = {truth}
        return truth

    @classmethod
    def multiple_ground_truth_reader(cls, filenames):
        truths = set()
        for filename in filenames:
            truths.add(cls.single_ground_truth_reader(filename, isset=False))
        return truths

    @BufferedGenerator.generator_method
    def groundtruth_paths_gen(self):
        truths = self.multiple_ground_truth_reader([adsb_file])
        yield None, truths


### Mimicing code from Kaggle...

In [ ]:
class ALVIRAReader_mod(DetectionReader):
    filename: str = Property(doc="Folder where scenario file is.")
    ndim_state: int = Property(default=6)
    pos_mapping: Tuple[int, int] = Property(default=(0, 2))
    vel_mapping: Tuple[int, int] = Property(default=(1, 3))
    pos_noise_diag: Tuple[float, float] = Property(
        default=(np.radians(1) ** 2, 25 ** 2))
    vel_noise_diag: Tuple[float, float] = Property(default=(1, 1))
    min_reflection: float = Property(default=-np.inf)
    max_reflection: float = Property(default=35)

    lat, lon, alt = 51.52126391, 5.85862734, 31

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        position_model = CartesianToBearingRange(
            self.ndim_state, self.pos_mapping, np.diag(self.pos_noise_diag),
            translation_offset=StateVector([*geodetic2enu(self.lat, self.lon, self.alt,
                                                          lat0, lon0, alt0)]))
        velocity_model = LinearGaussian(
            self.ndim_state, self.vel_mapping, np.diag(self.vel_noise_diag))

        self.model = CombinedReversibleGaussianMeasurementModel([position_model, velocity_model])

    @BufferedGenerator.generator_method
    def detections_gen(self):
        with open(self.filename, newline='') as csv_file:
            for row in csv.DictReader(csv_file):
                if not row['timestamp']:
                    continue

                timestamp = dateutil.parser.parse(row['timestamp'], ignoretz=True)
                lat = float(row['latitude'])
                lon = float(row['longitude']) 
                alt = float(row['altitude'])
                azimuth = np.radians(90 - float(row['az_velocity']))
                elevation = np.radians(float(row['alt_velocity']))
                speed = float(row['speed'])

                metadata = {
                    'classification': row['AlviraTracksTrack_Classification'],
                    'sensor': 'Alvira',
                    'reflection': float(row['AlviraTracksTrack_Reflection']),
                    'score': float(row['AlviraTracksTrack_Score']),
                }

                if not self.min_reflection < metadata['reflection'] < self.max_reflection:
                    continue

                easting, northing, *_ = geodetic2enu(lat, lon, alt, self.lat, self.lon, self.alt)
                rho, phi, _ = cart2sphere(easting, northing, alt)
                dx, dy, dz = sphere2cart(speed, azimuth, elevation)

                yield timestamp, {Detection(
                    [Bearing(phi), rho, dx, dy], timestamp=timestamp,
                    metadata=metadata, measurement_model=self.model)}

In [ ]:
alvira_file = '../../../data/ALVIRA_rdp.csv'
rdp_reader = RDPReader( # Elevation, bearing, range
    alvira_file,
    pos_noise_diag=[(np.pi/4)**2, np.radians(1)**2, 25**2], 
    vel_noise_diag=[1, 1, 1],
    min_reflection=-np.inf, max_reflection=35)

In [ ]:
alvira_file = '../../../data/ALVIRA_rdp.csv'
alvira = ALVIRAReader_mod( # Bearing, range
    alvira_file,
    pos_noise_diag=[np.radians(1)**2, 25**2], vel_noise_diag=[1, 1],
    min_reflection=-np.inf, max_reflection=35,
)

## Reform Alvira file to look like RDP
Adding in rho, theta, x, and y to make the Alvira file look like our RDP file.

In [ ]:
# from pymap3d import geodetic2enu
import pyproj
# geodesic = pyproj.Geod(ellps='WGS84')
from pymap3d import geodetic2enu

from stonesoup.functions import cart2sphere
from stonesoup.functions import pol2cart 
from math import pi

METERS_in_NM = 1852
lat, lon, alt = 51.52126391, 5.85862734, 31

alvira_df = pd.read_csv(alvira_file)
alvira_df['alt'] = alt

# Using code from Kaggle
alvira_df['translation_offset'] = alvira_df.apply(lambda x:
                                                  geodetic2enu(x.latitude, x.longitude, x.altitude, lat, lon, alt ),
                                                  axis=1)

alvira_df['easting'], alvira_df['northing'], alvira_df['upping'] = zip(*alvira_df['translation_offset'])

alvira_df['rho'], alvira_df['phi'], _ = zip(*alvira_df.apply(lambda x:
                                                            cart2sphere(x.easting, x.northing, x.altitude), 
                                                            axis=1)
                                            )


# alvira_df['theta'], alvira_df['back_theta'], alvira_df['distance'] = zip(*alvira_df.apply(lambda x:
#                                                                                 geodesic.inv( lon, lat, x.longitude, x.latitude),
#                                                                                 axis=1)
#                                                                     )

# alvira_df.loc[alvira_df.theta<0, 'theta'] = 2*pi + alvira_df.loc[alvira_df.theta<0, 'theta']
# alvira_df['theta_rad'] = np.deg2rad(alvira_df.theta)

# # alvira_df['rho'] = alvira_df['distance']/METERS_in_NM
# alvira_df['rho'] = alvira_df['distance']
# alvira_df.loc[alvira_df.theta<0, 'theta'] = 360 + alvira_df.loc[alvira_df.theta<0, 'theta']

# alvira_df['x'], alvira_df['y'] = zip(*alvira_df.apply(lambda p: pol2cart(p.rho, p.phi), axis=1))

alvira_df.to_csv(alvira_file, index=False)
alvira_df.loc[~alvira_df.latitude.isna(), ['latitude','longitude','easting','northing','upping','rho','phi']].head()


## Compare Stone Soup to my Coordinate Transforms

In [ ]:
from math import sin, cos
def polar_to_cartesian(rho, theta):
    """Convert polar coordinates to cartesian;
    Y = North, clockwise theta

    Given:
    rho: radius in NM
    theta: in degrees
    
    Return:
    x: NM
    y: NM
    """
    
    theta_rad = theta * pi/180
    x = rho * sin(theta_rad)
    y = rho * cos(theta_rad)

    return x, y

In [ ]:
import pyproj
geodesic = pyproj.Geod(ellps='WGS84')

alvira_df['theta'], alvira_df['back_theta'], alvira_df['distance'] = zip(*alvira_df.apply(
        lambda x: geodesic.inv( lon, lat, x.longitude, x.latitude),axis=1))

alvira_df.loc[alvira_df.theta<0, 'theta'] = 360 + alvira_df.loc[alvira_df.theta<0, 'theta']

alvira_df['x1'], alvira_df['y1'] = zip(*alvira_df.apply(lambda p: polar_to_cartesian(p.distance, p.theta), axis=1))

alvira_df['phi_deg'] = np.rad2deg(alvira_df.phi)

alvira_df.loc[~alvira_df.latitude.isna(), ['latitude','longitude','rho','phi','phi_deg','distance','theta','x1','y1']]


In [ ]:
# alvira_df.plot.scatter(x='latitude',y='longitude')
alvira_df.plot.scatter(x='x1',y='y1')
plt.grid()

In [ ]:
alvira_df.plot.scatter(x='longitude',y='latitude')
plt.grid()

In [ ]:
from stonesoup.plotter import Plotter
plotter = Plotter()

plotter.plot_measurements(
        [detection[1] for detection in alvira.detections_gen()], [0, 2])
        
plt.grid()

In [ ]:
rdp = RDPReader(rdp_file,
                pos_noise_diag=[np.radians(1)**2, 25**2], 
                vel_noise_diag=[1, 1],
                min_reflection=-np.inf, 
                max_reflection=np.inf
)

rdp_data.head()

In [ ]:
rdp_data['x_m'] = rdp_data.x * METERS_in_NM
rdp_data['y_m'] = rdp_data.y * METERS_in_NM

rdp_data.plot.scatter(x='x_m',y = 'y_m', color='red', marker="+")
plt.grid()

In [ ]:
plotter = Plotter()

plotter.plot_measurements(
        [detection[1] for detection in rdp.detections_gen()], 
        mapping=[0, 2],
        measurements_label='RDP', 
        color="red",
        alpha=0.3,
        marker="+",
        zorder=10)
plt.grid()

In [ ]:
def generate_timestamps(start_time, end_time):
    total_seconds = (end_time - start_time).total_seconds()
    return [start_time + timedelta(seconds=n) for n in range(ceil(total_seconds))]

In [ ]:
adsb = ADSBTruthReader(adsb_file)
ground_truth = set()
for time, truths in adsb:
    ground_truth.update(truths)


In [ ]:
plotter = Plotter()
plotter.plot_ground_truths(ground_truth, [0, 2], color='blue', marker='s', markerfacecolor='none', alpha=0.3)
plt.grid()

In [ ]:
# ground_truth=True

timestamps = generate_timestamps(rdp_data['timestamp'].min(), rdp_data['timestamp'].max())

plotter = AnimatedPlotterly(timestamps, tail_length=0.3, sim_duration=1, equal_size=True)
plotter.fig.update_layout(width=800, height=800)
rdp_measurements = []

for detection in [detection[1] for detection in rdp.detections_gen()]:
    rdp_measurements.append(detection)

plotter.plot_measurements(rdp_measurements,
                          mapping=[0, 2],
                          measurements_label='RDP',
                          marker=dict(color='rgba(255, 0, 0, 0.7)',
                                      size=5, 
                                      symbol="cross")                
)

plotter.plot_ground_truths(ground_truth, 
                           mapping=[0, 2], 
                           mode='markers', 
                           marker=dict(color='rgba(0, 0, 255, 0.2)',
                                      size=5, 
                                      symbol="square-open")
                            )

plotter.fig

### Create Predictor and Updater

Create our model used for prediction, using 3 CV models, making state space $ x, \dot x, y, \dot y, z, \dot z $; where $x$ is east, $y$ is north, and $z$ is altitude.

In [ ]:
transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(40)]*3)
predictor = ExtendedKalmanPredictor(transition_model)

Create our updater. We won't define a model here, as we'll used the ones assigned to detections.

In [ ]:
updater = ExtendedKalmanUpdater(measurement_model=None)

### Create Hypothesiser and Data Associator

This create hypothesier and data associator componets used for associating tracks and detections. Also create slightly stricter versions of these to limit track initiation.

In [ ]:
from typing import Tuple

import numpy as np
from stonesoup.hypothesiser.distance import DistanceHypothesiser
from stonesoup.models.measurement.nonlinear import CombinedReversibleGaussianMeasurementModel
from stonesoup.gater.base import Gater
from stonesoup.base import Property

class SensorLocationGater(Gater):

    hypothesiser: DistanceHypothesiser = Property(
        doc='hypothesiser to use when far enough away from sensors')
    pos_mapping: Tuple[int, int] = Property(default=(0, 2))
    min_distance_from_sensor: float = Property(default=80)

    def hypothesise(self, track, detections, timestamp, *args, **kwargs):
        for detection in detections:
            measurement_model = detection.measurement_model
            if isinstance(measurement_model, CombinedReversibleGaussianMeasurementModel):
                measurement_model = measurement_model.model_list[0]
            sensor_location = measurement_model.translation_offset[:, 0][:len(self.pos_mapping)]

            track_location = track.state_vector[self.pos_mapping, 0]
            difference = track_location - sensor_location
            euclidean_dist = np.sqrt(difference[0] ** 2 + difference[1] ** 2)

            if euclidean_dist <= self.min_distance_from_sensor:
                return self.hypothesiser.hypothesise(track, set(), timestamp)
        return self.hypothesiser.hypothesise(track, detections, timestamp)


In [ ]:
hypothesiser = DistanceHypothesiser(predictor, updater, measure=Mahalanobis(), missed_distance=4)

# This is reducing missed distance to 2 units of Mahalanobis i.e. std. deviation, for initialisation
init_hypothesiser = DistanceHypothesiser(predictor, updater, measure=Mahalanobis(), missed_distance=2)

# This will ignore sensor data when drone within min. distance. Avoids bad bearing/elevation measurements
# causing issues in particular with rapid changes in velocity.
hypothesiser = SensorLocationGater(hypothesiser, min_distance_from_sensor=80)
init_hypothesiser = SensorLocationGater(init_hypothesiser, min_distance_from_sensor=80)

data_associator = GNNWith2DAssignment(hypothesiser)
init_data_associator = GNNWith2DAssignment(init_hypothesiser)

### Create Track Initiators and Deleters

The deleters in this example will remove tracks where no detections have been associated for a period of time,
or uncertainty in position has grown too large.

In [ ]:
deleter = CompositeDeleter(
    [
        UpdateTimeDeleter(time_since_update=timedelta(seconds=30), delete_last_pred=True),
        CovarianceBasedDeleter(covar_trace_thresh=5000, mapping=[0, 2], delete_last_pred=True),
    ],
    intersect=False)

# More aggressive deletion when trying to initalise a track
init_deleter = CompositeDeleter(
    [
        UpdateTimeDeleter(time_since_update=timedelta(seconds=15), delete_last_pred=True),
        CovarianceBasedDeleter(covar_trace_thresh=3000, mapping=[0, 2], delete_last_pred=True),
    ],
    intersect=False)

Our initiator will use the detections position/velocity from the radar for easting/northing $x$/$y$ (hence just leaving prior state vector and covariance `0` for those elements), but use a prior value for altitude where 2D Radar initialises the track.

In [ ]:
prior = GaussianState([0, 0, 4000, 0, 0, 0], np.diag([0, 0, 0, 0, 1000, 100]))

base_initiator = MultiMeasurementInitiator(
    prior_state=prior,
    measurement_model=None,
    deleter=init_deleter,
    data_associator=init_data_associator,
    updater=updater,
    min_points=4,
)

In [ ]:
from stonesoup.base import Property
from stonesoup.initiator import Initiator

class rdp_initiator(Initiator):

    initiator: Initiator = Property()

    def initiate(self, detections, timestamp, **kwargs):
        for detection in detections:
            # if detection.metadata['sensor'] == 'RDP':
            return self.initiator.initiate(detections, timestamp, **kwargs)
        return self.initiator.initiate(set(), timestamp, **kwargs)

## Run Tracker and Plot Results

In [ ]:
detector = MultiDataFeeder([rdp])
initiator = rdp_initiator(base_initiator)

tracker = MultiTargetTracker(
    detector=detector,
    initiator=initiator,
    deleter=deleter,
    data_associator=data_associator,
    updater=updater,
)

In [ ]:
tracks = set()
detections = set()
try:
    for time, ctracks in tracker:
        tracks |= ctracks
        detections |= tracker.detector.detections
except:
    pass
len(tracks)

In [ ]:
len(detections)

In [ ]:
from stonesoup.plotter import Plotter

plotter = Plotter()
# sensor='Alvira'
plotter.plot_measurements(
        {detection for detection in detections}, 
        mapping=[0, 2],
        measurements_label='RDP', 
        color='red', 
        marker='+',
        alpha=0.3,
        zorder=10)
plotter.plot_ground_truths(ground_truth, [0, 2], color='black')
plotter.plot_tracks(tracks, [0,2], uncertainty=True)
for name, position in sensor_positions.items():
    plotter.ax.scatter(*position, label=name, marker='X')
    plotter.ax.text(*position+20, name.title())
track_xlim = plotter.ax.get_xlim()
track_ylim = plotter.ax.get_ylim()